<a href="https://colab.research.google.com/github/sukritichawla/legaldocs_ST/blob/main/notebooks/01_simplification_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 1. GPU check
!nvidia-smi

# 2. Install dependencies
!pip -q install -U transformers datasets sentencepiece accelerate huggingface_hub python-dotenv feedparser beautifulsoup4 sacrebleu rouge-score gradio



# 3. Clone or pull repo
from getpass import getpass
token = getpass("Enter your GitHub PAT: ")
USERNAME = "sukritichawla"
REPO = "legaldocs_ST"
REPO_URL = f"https://{token}@github.com/{USERNAME}/{REPO}.git"
!git clone $REPO_URL


# 4. Go inside repo
%cd /content/legaldocs_ST

# 5. Hugging Face login
from huggingface_hub import login
login()


Thu Sep  4 05:48:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             29W /   70W |    2750MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
from transformers import pipeline

# Load a summarization model
simplifier = pipeline("summarization", model="facebook/bart-large-cnn")

# Example legal text
text = """Whereas it is expedient to provide for the extension of loans to
students pursuing higher education under certain conditions..."""

# Run simplification
result = simplifier(text, max_length=80, min_length=30, do_sample=False)
print("Simplified English:", result[0]["summary_text"])


Device set to use cuda:0
Your max_length is set to 80, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


Simplified English: It is expedient to provide for the extension of loans to students pursuing higher education under certain conditions. Whereas it is  expedient  to provide  loans to  students pursuing higher education under certain conditions.


In [2]:
simplifier_t5 = pipeline("summarization", model="google/flan-t5-base")

result_t5 = simplifier_t5(text, max_length=80, min_length=30, do_sample=False)
print("Simplified English (FLAN-T5):", result_t5[0]["summary_text"])


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Your max_length is set to 80, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Simplified English (FLAN-T5): In the case of loans, it is expedient to provide for the extension of loans to students pursuing higher education under certain conditions. Moreover, if a loan is extended to a student pursuing a higher education degree under certain circumstances, the student must be able to repay the loan in full.


In [9]:
!pwd

/content/legaldocs_ST


In [10]:
import pandas as pd

examples = [text]
outputs = [result[0]["summary_text"]]

df = pd.DataFrame({"original": examples, "simplified": outputs})
df.to_csv("data/processed/simplified_samples.csv", index=False)
print("Saved to data/processed/simplified_samples.csv")


Saved to data/processed/simplified_samples.csv
